In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import os,sys

In [22]:

mnist = input_data.read_data_sets('data',one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size
print(n_batch)

#图片数量
image_num = 3000
#载入图片
DIR = "/Users/simplzy/Desktop/deep_learning/jupyter_notebook/MNIST/"
tf.reset_default_graph()
sess = tf.Session() 

embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name = 'embedding')

def variable_summaries(var):
    mean = tf.reduce_mean(var) 
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev',stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)


with tf.name_scope('input'):
    x = tf.placeholder(shape=[None, 784], dtype= tf.float32,name="input-x")
    y = tf.placeholder(shape=[None, 10], dtype = tf.float32,name="input-y")
    #定义x,y,keep_prob'
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")

with tf.name_scope('reshape'):
    image_reshape_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_reshape_input, 10)
    
#构建3层神经网络
with tf.name_scope('layer'):
    with tf.name_scope('weights1'):
        W1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1), dtype=tf.float32)
        variable_summaries(W1)
    with tf.name_scope('biases1'):
        b1 = tf.Variable(tf.zeros([500]) + 0.1)
        variable_summaries(b1)
    L1 = tf.tanh(tf.matmul(x, W1) + b1)
    L1_drop = tf.nn.dropout(L1, keep_prob)
    
    with tf.name_scope('weights2'):
        W2 = tf.Variable(tf.truncated_normal([500, 300], stddev=0.1), dtype=tf.float32)
        variable_summaries(W2)
    with tf.name_scope('biases2'):
        b2 = tf.Variable(tf.zeros([300]) + 0.1)
        variable_summaries(b2)
    L2 = tf.tanh(tf.matmul(L1_drop, W2) + b2)
    L2_drop = tf.nn.dropout(L2, keep_prob)

    with tf.name_scope('weights3'):
        W3 = tf.Variable(tf.truncated_normal([300, 10], stddev=0.1), dtype=tf.float32)
        variable_summaries(W3)
    with tf.name_scope('biases3'):
        b3 = tf.Variable(tf.zeros([10]) + 0.1)
        variable_summaries(b3)
        
prediction = tf.nn.softmax(tf.matmul(L2_drop, W3) + b3)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y, logits= prediction))
    tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)


with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        

sess.run(tf.global_variables_initializer())

if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector')
    os.mkdir(DIR + 'projector/projector')
    #os.remove(DIR + 'projector/projector/metadata.tsv')
#os.system('rm ' + DIR + 'projector/projector/*')
with open('projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')
    
merged = tf.summary.merge_all()
projector_writer = tf.summary.FileWriter(DIR + 'projector/projector', sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer, config)



#writer = tf.summary.FileWriter('logs/', sess.graph)
count = 0
for epoch in range(50):
    for j in range(n_batch):
        count += 1
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.8},\
                              options=run_options,run_metadata=run_metadata)
        projector_writer.add_run_metadata(run_metadata, 'step%04d' % count)
        projector_writer.add_summary(summary, count)
    acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob : 1.0})   
    print ("Iter " + str(epoch) + ", Testing Accuracy= " + str(acc))

saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step = count)
projector_writer.close()
sess.close()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
550
Iter 0, Testing Accuracy= 0.9424
Iter 1, Testing Accuracy= 0.9548
Iter 2, Testing Accuracy= 0.9608
Iter 3, Testing Accuracy= 0.9682
Iter 4, Testing Accuracy= 0.9677
Iter 5, Testing Accuracy= 0.9697
Iter 6, Testing Accuracy= 0.9725
Iter 7, Testing Accuracy= 0.9729
Iter 8, Testing Accuracy= 0.973
Iter 9, Testing Accuracy= 0.9734
Iter 10, Testing Accuracy= 0.9744
Iter 11, Testing Accuracy= 0.9737
Iter 12, Testing Accuracy= 0.9749
Iter 13, Testing Accuracy= 0.9738
Iter 14, Testing Accuracy= 0.9711
Iter 15, Testing Accuracy= 0.9779
Iter 16, Testing Accuracy= 0.9748
Iter 17, Testing Accuracy= 0.9775
Iter 18, Testing Accuracy= 0.9772
Iter 19, Testing Accuracy= 0.9783
Iter 20, Testing Accuracy= 0.9777
Iter 21, Testing Accuracy= 0.9762
Iter 22, Testing Accuracy= 0.9784
Iter 23, Testing Accuracy= 0.9759
Iter 24, Testing Accu